In [ ]:
# ==================================================================================
# Diplomado en Estadística Aplicada a la Toma de Decisiones con Lenguaje R y Python
# Universidad Privada Boliviana
# ----------------------------------------------------------------------------------
#   MODELOS PREDICTIVOS PARA LA TOMA DE DECISIONES ESTRATEGICAS
# ----------------------------------------------------------------------------------
#         Enrique Alejandro Laurel Cossio, Septiembre 2024
# ==================================================================================
#                Introducción a Redes Neuronales
# ==================================================================================

In [44]:
# Cargamos Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,confusion_matrix,recall_score,classification_report,precision_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

# redes neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.regularizers import l2

# 1. REDES NEURONALES - REGRESIÓN

In [17]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS/main/data/insurance.csv'
seguros = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(seguros.shape) #
seguros.head(2)

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [18]:
# una copia de la fuente de datos
df = seguros.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [19]:
# a dummies las variables categoricas
df = pd.get_dummies(df, columns=['region','sex','smoker'], drop_first=True)
print(df.shape)
df.head(2)

(1338, 9)


,age,bmi,children,charges,region_northwest,region_southeast,region_southwest,sex_male,smoker_yes
0,19,27.90,0,16884.9240,False,False,True,False,True
1,18,33.77,1,1725.5523,False,True,False,True,False


In [20]:
# variables dummies  a entero
df = df.replace({True: 1, False: 0})

In [21]:
# Definir las variables independientes y dependientes
X = df[['age', 'sex_male', 'bmi', 'children', 'smoker_yes', 'region_northwest','region_southeast','region_southwest']]
y = df['charges'] # variables dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) # relacionado intercepto

In [22]:
# División de conjunto de datos en entrenamiento y testeo
# train - test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 22) #Separamos 30% para test

In [24]:
# Crear el modelo
model_RNA_Regres = Sequential()
model_RNA_Regres.add(Dense(units=64, activation='relu', input_dim=8))  # Capa de entrada
model_RNA_Regres.add(Dense(units=32, activation='relu'))  # Primera capa oculta
model_RNA_Regres.add(Dense(units=16, activation='relu'))  # Segunda capa oculta
model_RNA_Regres.add(Dense(units=1, activation='linear'))  # Capa de salida (regresion)

# Compilar el modelo
model_RNA_Regres.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Entrenar el modelo
model_RNA_Regres.fit(x_train.drop(columns=['const']), y_train, epochs=10, batch_size=32)

# Evaluar el rendimiento del modelo
loss, mae = model_RNA_Regres.evaluate(x_train.drop(columns=['const']), y_train)
print(f"Pérdida: {loss:.4f}, Error Cuadratico Medio: {mae:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 316406176.0000 - mean_squared_error: 316406176.0000
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 336649312.0000 - mean_squared_error: 336649312.0000 
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 321900672.0000 - mean_squared_error: 321900672.0000
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 301815424.0000 - mean_squared_error: 301815424.0000
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 301791200.0000 - mean_squared_error: 301791200.0000
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 272370816.0000 - mean_squared_error: 272370816.0000 
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 194456576.0000 - mean_squared_error: 194456576.0000
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 158450640.0000 - mean_squared_error: 158450640.0000
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 123003912.0000 - mean_squared_error: 123003912.0000
Epoch 10

In [25]:
# Ajustar el modelo de regresión lineal
modelo_OLS = sm.OLS(y_train, x_train).fit()

# Resumen del modelo
print(modelo_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     354.2
Date:                Fri, 27 Sep 2024   Prob (F-statistic):          9.23e-276
Time:                        14:32:11   Log-Likelihood:                -9456.6
No. Observations:                 936   AIC:                         1.893e+04
Df Residuals:                     927   BIC:                         1.897e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.299e+04   1148.340  

In [26]:
# ajustamos el indice en la data de test
x_test.reset_index(drop=True, inplace=True)

In [28]:
# calculamos predicción

# Regresión Lineal (Mínimos cuadrados ordinarios)
print('Predicción Regresión Lineal - OLS')
y_pred_OLS = modelo_OLS.predict(x_test)
print(y_pred_OLS.head(2))

# RNA
print('Predicción RNA - Regresión')
y_pred_RNA = model_RNA_Regres.predict(x_test.drop(columns='const'))
y_pred_RNA = pd.DataFrame(y_pred_RNA)
print(y_pred_RNA.head(2))

Predicción Regresión Lineal - OLS
0    22825.346680
1    17845.585097
dtype: float64
Predicción RNA - Regresión
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
              0
0   8278.173828
1  19579.792969


In [29]:
# Error Cuadratico Medio

# Regresión Lineal - OLS
mse_OLS = mean_squared_error(y_test, y_pred_OLS)
print(f"Error cuadrático medio (MSE)- Regresión Lineal OLS: {mse_OLS:.4f}")

# RNA - Regresión
mse_RNA = mean_squared_error(y_test, y_pred_RNA)
print(f"Error cuadrático medio (MSE)- RNA: {mse_RNA:.4f}")
# Mientras menos mejor

Error cuadrático medio (MSE)- Regresión Lineal OLS: 40986218.2527
Error cuadrático medio (MSE)- RNA: 143394191.6992


# 2. REDES NEURONALES - CLASIFICACIÓN

In [36]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS/main/data/ifood_df.csv'
ifood_df = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(ifood_df.shape) #
ifood_df.head(2)

(2205, 39)


,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0


In [37]:
# una copia a los datos
df = ifood_df.copy()

In [38]:
# seleccionamos variables explicativas y variables dependiente
X = df[['Kidhome','Recency','education_Basic','Complain','Income','MntTotal','Age']]
y = df['Response']

In [39]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [50]:
# Entrenamiento Modelos de Redes Neuronales Artificiales
# Crear el modelo
model_RNA_Clasif = Sequential()
model_RNA_Clasif.add(Dense(units=64, activation='relu', input_dim=7,kernel_regularizer=l2(0.01)))  # Capa de entrada
model_RNA_Clasif.add(Dense(units=32, activation='relu',kernel_regularizer=l2(0.01)))  # Primera capa oculta
model_RNA_Clasif.add(Dense(units=16, activation='relu',kernel_regularizer=l2(0.01)))  # Segunda capa oculta
model_RNA_Clasif.add(Dense(units=1, activation='sigmoid'))  # Capa de salida (clasificación binaria)

# Compilar el modelo
model_RNA_Clasif.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision'])

# Entrenar el modelo
model_RNA_Clasif.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluar el rendimiento del modelo
loss, accuracy = model_RNA_Clasif.evaluate(X_train, y_train)
print(f"Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 196.4492 - precision: 0.1570
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 62.1481 - precision: 0.1369
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 64.4970 - precision: 0.1292
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.2474 - precision: 0.1680
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 59.7410 - precision: 0.1671
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.7492 - precision: 0.2778
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.4255 - precision: 0.2309
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.9499 - precision: 0.2366
Epoch 9/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.0340 - precision: 0.1912
Epoch 10/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 42.1235 - precision: 0.1748
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.9005 - precision: 0.0000e+00
Pérdida: 10.6313, Precisión: 0.0000


In [47]:
# Árboles de Decisión
# hallamos los mejores hiperparametros del modelo
# Crear el clasificador
clf = DecisionTreeClassifier()

# Definir la grilla de hiperparámetros a explorar
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar búsqueda en la grilla
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
best_params

{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [48]:
# Entrenamos el Modelo de Árbol de Decisión para Clasificación

# Crea un clasificador de árbol de decisión
modelo_ArbolDecisionClasif = DecisionTreeClassifier(max_depth=10,min_samples_leaf=2,min_samples_split=2)

# Entrena el modelo en los datos de entrenamiento
modelo_ArbolDecisionClasif.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=2)

In [ ]:
## Obtenemos la importancia de las variables
#feature_importance = modelo_ArbolDecisionClasif.feature_importances_
#
## Creamos un DataFrame para visualizar las importancias junto con los nombres de las variables
#feature_names = X_train.columns
#importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
#
## Ordenamos por importancia descendente
#importance_df = importance_df.sort_values(by='Importance', ascending=False)
#
## Graficamos la importancia de las variables
#plt.figure(figsize=(10, 6))
#plt.barh(importance_df['Feature'], importance_df['Importance'])
#plt.xlabel('Importance')
#plt.ylabel('Feature')
#plt.title('Importancia de Variables en un Modelo de Árbol de Decisión - Clasificación')
#plt.show()

In [ ]:
#import matplotlib.pyplot as plt
#from sklearn.tree import plot_tree
#plt.figure(figsize=(10, 6))
#plot_tree(modelo_ArbolDecisionClasif, filled=True, feature_names=X_train.columns)
#plt.show()

In [49]:
# ajustamos el indice en x_test
X_test.reset_index(drop=True, inplace=True)

In [52]:
# Realiza predicciones en el conjunto de datos de prueba

# RNA
print('Predicción RNA')
y_pred_RNA_Clasif = model_RNA_Clasif.predict(X_test)
y_pred_RNA_Clasif = pd.DataFrame(y_pred_RNA_Clasif)
print(y_pred_RNA_Clasif.head(2))

# Arbol de Decisión - Clasificación
print('Predicción Árbol de Decisión - Clasificación')
y_pred_ArbolDecisionClasif = modelo_ArbolDecisionClasif.predict_proba(X_test)
y_pred_ArbolDecisionClasif = pd.DataFrame(y_pred_ArbolDecisionClasif)
y_pred_ArbolDecisionClasif = y_pred_ArbolDecisionClasif.iloc[:,1]
print(y_pred_ArbolDecisionClasif.head(2))

Predicción RNA
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
              0
0  0.000000e+00
1  3.223526e-36
Predicción Árbol de Decisión - Clasificación
0    0.0
1    0.0
Name: 1, dtype: float64


In [55]:
# Obteniendo el mejor corte de predicción
# Calcular la curva ROC - RNA
fpr, tpr, thresholds = roc_curve(y_test, y_pred_RNA_Clasif)

# Encontrar el umbral óptimo (máximo índice de Youden)
youden_index = tpr - fpr
optimal_threshold_RNA = thresholds[np.argmax(youden_index)]

print("Umbral óptimo Logit:", optimal_threshold_RNA)

Umbral óptimo Logit: 3.1820968e-28


In [56]:
# Obteniendo el mejor corte de predicción
# Calcular la curva ROC - Arbol de Decisión
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ArbolDecisionClasif)

# Encontrar el umbral óptimo (máximo índice de Youden)
youden_index = tpr - fpr
optimal_threshold_AD = thresholds[np.argmax(youden_index)]

print("Umbral óptimo- Arbol de Decisión:", optimal_threshold_AD)

Umbral óptimo- Arbol de Decisión: 0.08571428571428572


In [57]:
# Matriz de confusión

            #Predicción
            #   0   1
#Valor Real 0 # VN FP
#Valor Real 1 # FN VP

# Regresión Logistica - RNA
confusion = confusion_matrix(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Matriz de confusión - RNA:")
print(confusion)

# Árbol de Decisión - Clasificación
confusion = confusion_matrix(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("\nMatriz de confusión - Árbol de Decisión:")
print(confusion)

Matriz de confusión - RNA:
[[292  70]
 [ 46  33]]

Matriz de confusión - Árbol de Decisión:
[[279  83]
 [ 43  36]]


In [58]:
# Exactitud o Accuracy del modelo

# RNA
exactitud_test_RNA = accuracy_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Precisión del modelo-RNA:", exactitud_test_RNA)

# Árbol de Decisión - Clasificación
exactitud_test_ArbolDecisionClasif = accuracy_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Precisión del modelo- Árboles de Decisión:", exactitud_test_ArbolDecisionClasif)

Precisión del modelo-RNA: 0.7369614512471655
Precisión del modelo-Arbol de Decisión: 0.7142857142857143


In [60]:
# Sensibilidad (Proporción de verdaderos positivos de los valores reales positivos)

# RNA
sensibilidad_test_RNA = recall_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Sensibilidad - RNA:", sensibilidad_test_RNA)

# Árbol de Decisión - Clasificación
sensibilidad_test_ArbolDecisionClasif = recall_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Sensibilidad - Arbol de Decisión:", sensibilidad_test_ArbolDecisionClasif)

Sensibilidad - RNA: 0.4177215189873418
Sensibilidad - Arbol de Decisión: 0.45569620253164556


In [61]:
# Precisión (Proporción de Verdaderos Positivos de las predicciónes positivas del modelo)

# RNA
precision_score_RNA = precision_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Precisión del modelo - RNA", precision_score_RNA)

# Arbol de Decisión - Clasificación
precision_score_ArbolDecisionClasif = precision_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Precisión del modelo - Árbol de Decisión", precision_score_ArbolDecisionClasif)

Precisión del modelo - RNA 0.32038834951456313
Precisión del modelo - Árbol de Decisión 0.3025210084033613


# 3. RESULTADOS

In [62]:
print('Nro de clientes en el piloto',df.shape[0])

Nro de clientes en el piloto 2205


In [63]:
costo_piloto=6720
costo_piloto

6720

In [64]:
ingreso_piloto=3674
ingreso_piloto

3674

In [65]:
# beneficio del piloto
ingreso_piloto-costo_piloto

-3046

In [ ]:
# Propuesta

In [66]:
# calculamos el ingreso por cliente
print('Clientes que aceptaron la oferta',df[df['Response']==1].shape[0],'de',df.shape[0],'clientes')
ingreso_xcliente = ingreso_piloto/df[df['Response']==1].shape[0]
print('ingreso por cliente que aceptó la oferta',ingreso_xcliente)

Clientes que aceptaron la oferta 333 de 2205 clientes
ingreso por cliente que aceptó la oferta 11.033033033033034


In [67]:
# costo por cliente
costo_xcliente = costo_piloto/df.shape[0]
print('Costo por cliente',costo_xcliente)

Costo por cliente 3.0476190476190474


In [85]:
# RNA
confusion_RNA = confusion_matrix(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Matriz de confusión - RNA:")
print(confusion_RNA)

# Extrae los valores
VN_RNA,  FP_RNA,FN_RNA, VP_RNA = confusion_RNA.ravel()

print(f"Verdaderos Negativos (VN): {VN_RNA}")
print(f"Falsos Positivos (FP): {FP_RNA}")
print(f"Falsos Negativos (FN): {FN_RNA}")
print(f"Verdaderos Positivos (VP): {VP_RNA}")

Matriz de confusión - RNA:
[[292  70]
 [ 46  33]]
Verdaderos Negativos (VN): 292
Falsos Positivos (FP): 70
Falsos Negativos (FN): 46
Verdaderos Positivos (VP): 33


In [86]:
# Árbol de Decisión - Clasificación
confusion_DT = confusion_matrix(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("\nMatriz de confusión - Árbol de Decisión:")
print(confusion_DT)

# Extrae los valores
VN_DT, FP_DT, FN_DT, VP_DT = confusion_DT.ravel()

print(f"Verdaderos Negativos (VN): {VN_DT}")
print(f"Falsos Positivos (FP): {FP_DT}")
print(f"Falsos Negativos (FN): {FN_DT}")
print(f"Verdaderos Positivos (VP): {VP_DT}")


Matriz de confusión - Árbol de Decisión:
[[279  83]
 [ 43  36]]
Verdaderos Negativos (VN): 279
Falsos Positivos (FP): 83
Falsos Negativos (FN): 43
Verdaderos Positivos (VP): 36


In [98]:
# RNA
# inversión o costo - RNA, datos de testeo, representan un 20% del total de clientes en piloto

# numero de clientes con alta probabilidad
cli_test_alta_prob_RNA = FP_RNA+VP_RNA
print('Clientes Test con alta probabilidad',cli_test_alta_prob_RNA)

# beneficio esperado en clientes de datos de test con alta probabilidad
benef_cli_test_alta_prob_RNA=ingreso_xcliente*VP_RNA-costo_xcliente*(FP_RNA+VP_RNA)
print('Beneficio Cli_test con alta probabilidad($us)',benef_cli_test_alta_prob_RNA)

Clientes Test con alta probabilidad 103
Beneficio Cli_test con alta probabilidad($us) 50.18532818532822


In [90]:
# proporcion de clientes que se enviara la campaña
#(Predicción Positiva entre el total de clientes)
prop_campania_RNA=(VP_RNA+FP_RNA)/(VP_RNA+FP_RNA+VN_RNA+FN_RNA)
prop_campania_RNA

0.23356009070294784

In [93]:
# bajo el supuesto que el piloto es una muestra representativa de la población
# bajo el supuesto que la poblacion de clientes es 2,000,000
pob_clientes=2000000
print('Población de Clientes',pob_clientes)

# Se envia la campaña de marketing unicamente al 23% de la población que calificaran como muy probables a aceptar la oferta
cli_campania_RNA = pob_clientes*prop_campania_RNA
print('Clientes que recibiran la campaña de marketing',cli_campania_RNA)

Población de Clientes 2000000
Clientes que recibiran la campaña de marketing 467120.1814058957


In [122]:
# Beneficios Esperados con modelo RNA
# usando la siguiente formula
# donde :
# N: población calificable para la campaña (cli_campania_RNA)
# n: población calificable en el subconjunto de testeo,muestra de la población(cli_test_alta_prob_RNA)
# Factor de expansión = N/n
# Benef_test: Beneficio en el grupo de testeo (benef_cli_test_alta_prob_RNA)
# Asi el beneficio esperado en la población calificable para la campaña es:  Benef_test* (N/n)
print('Beneficio esperado de la campaña en población calificada(alta probabilidad), $us',(benef_cli_test_alta_prob_RNA)*(cli_campania_RNA/cli_test_alta_prob_RNA))

Beneficio esperado de la campaña en población calificada(alta probabilidad), $us 227597.86025092163
